<a href="https://colab.research.google.com/github/mshinohar/langchain-book/blob/main/udemy_langchain_LCEL_copy_Ch14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ch14 LCEL
- Udemy講座「LangChainによるLLM App開発入門」セクション
- 「（アップデート）LangChain Expression Language （LCEL）」のソースコード

## LangChain Expression Language （LCEL）の概要と実装例

In [1]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [2]:
!pip install --quiet langchain==0.0.329 openai==0.28.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

# PromptTemplateを準備
prompt = PromptTemplate.from_template("""料理のレシピを考えてください。

料理名: {dish}""")

# LLMを準備
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Chainを作成
chain = prompt | llm

# Chainを作成する処理は、LCEL以前は以下のように書いていました。
# chain = LLMChain(llm=chat, prompt=prompt)

result = chain.invoke({"dish": "カレー"})
print(result.content)

材料:
- 400gの鶏肉（もも肉や胸肉など、お好みの部位）
- 2個の玉ねぎ
- 2個のにんじん
- 2個のじゃがいも
- 2個のトマト
- 3つのにんにくのかけら
- 1つの生姜のかけら
- 2つのカレールウ（お好みの辛さ）
- 2カップの水
- 2つのカレーパウダー
- 2つのターメリックパウダー
- 2つのコリアンダーパウダー
- 2つのクミンパウダー
- 2つのパプリカパウダー
- 2つの塩
- 2つの砂糖
- 2つの植物油

手順:
1. 鶏肉を食べやすい大きさに切り、玉ねぎ、にんじん、じゃがいも、トマトもそれぞれ適切な大きさに切る。
2. 鍋に植物油を熱し、にんにくと生姜を加えて香りが立つまで炒める。
3. 鶏肉を加えて全体が白くなるまで炒める。
4. 野菜を加えて炒め、野菜が少し柔らかくなるまで続ける。
5. カレールウを加えてよく混ぜ、香りが立つまで炒める。
6. 水を加え、煮立つまで火にかける。
7. カレーパウダー、ターメリックパウダー、コリアンダーパウダー、クミンパウダー、パプリカパウダー、塩、砂糖を加えてよく混ぜる。
8. 中火で約30分間煮込む。必要に応じて水を追加する。
9. カレーがとろみがついて野菜が柔らかくなったら、完成です。
10. ご飯やナンと一緒にお召し上がりください。お好みでコリアンダーの葉やライムをトッピングしても美味しいです。


In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field

# OutputParserを準備
class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")

output_parser = PydanticOutputParser(pydantic_object=Recipe)

# PromptTemplateを準備
prompt = PromptTemplate.from_template("""料理のレシピを考えてください。

{format_instructions}

料理名: {dish}""", partial_variables = {"format_instructions": output_parser.get_format_instructions()})

# LLMを準備
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Chainを作成
chain = prompt | llm | output_parser

# Chainを実行
result = chain.invoke({"dish": "カレー"})
print(type(result))
print(result)

<class '__main__.Recipe'>
ingredients=['玉ねぎ', 'にんじん', 'じゃがいも', '牛肉', 'カレールー', '水'] steps=['玉ねぎ、にんじん、じゃがいもを切ります。', '牛肉を炒めます。', '野菜を加えて炒めます。', '水を加えて煮込みます。', 'カレールーを加えて溶かします。', '煮込んで完成です。']
